In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
from src.inference import get_feature_store

In [11]:
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]

2025-03-06 01:27:02,114 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-06 01:27:02,119 INFO: Initializing external client
2025-03-06 01:27:02,120 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-06 01:27:03,118 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215672
Fetching data from 2025-02-05 06:27:02.114480+00:00 to 2025-03-06 05:27:02.114480+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (6.61s) 


In [12]:
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)

,pickup_hour,pickup_location_id,rides
0,2025-02-05 07:00:00+00:00,2,0
1,2025-02-05 08:00:00+00:00,2,0
2,2025-02-05 09:00:00+00:00,2,0
3,2025-02-05 10:00:00+00:00,2,0
4,2025-02-05 11:00:00+00:00,2,0
...,...,...,...
175135,2025-03-06 01:00:00+00:00,263,6
175136,2025-03-06 02:00:00+00:00,263,6
175137,2025-03-06 03:00:00+00:00,263,4
175138,2025-03-06 04:00:00+00:00,263,6


In [13]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 175140 entries, 0 to 181165
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype                  
---  ------              --------------   -----                  
 0   pickup_hour         175140 non-null  datetime64[us, Etc/UTC]
 1   pickup_location_id  175140 non-null  int32                  
 2   rides               175140 non-null  int32                  
dtypes: datetime64[us, Etc/UTC](1), int32(2)
memory usage: 4.0 MB


In [14]:
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

In [15]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 175140 entries, 0 to 181165
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         175140 non-null  datetime64[us]
 1   pickup_location_id  175140 non-null  int32         
 2   rides               175140 non-null  int32         
dtypes: datetime64[us](1), int32(2)
memory usage: 4.0 MB


In [16]:
from src.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)

In [17]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,2,0,0,0,3,0,0,5,1,4,...,1,0,0,0,0,0,0,0,255,2025-03-05 07:00:00
1,2,1,0,0,1,1,0,1,0,1,...,1,0,0,0,0,0,3,0,218,2025-03-05 07:00:00
2,369,273,249,21,251,348,17,373,286,251,...,61,7,28,7,1,2,5,44,237,2025-03-05 07:00:00
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,57,2025-03-05 07:00:00
4,0,0,0,0,0,1,0,1,0,0,...,1,0,0,0,0,0,0,2,56,2025-03-05 07:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,0,2,0,0,4,0,1,0,0,2,...,0,0,0,0,0,1,1,1,71,2025-03-05 07:00:00
248,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,8,2025-03-05 07:00:00
249,0,1,1,1,1,2,0,4,0,1,...,0,1,0,0,1,1,1,4,168,2025-03-05 07:00:00
250,11,287,206,131,101,199,173,84,61,187,...,91,32,36,13,6,8,51,106,186,2025-03-05 07:00:00


In [18]:
from src.inference import load_batch_of_features_from_store
current_date = pd.Timestamp.now(tz='Etc/UTC')
features = load_batch_of_features_from_store(current_date)

2025-03-06 01:27:58,781 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-06 01:27:58,804 INFO: Initializing external client
2025-03-06 01:27:58,805 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-06 01:27:59,749 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215672
Fetching data from 2025-02-05 06:27:58.781180+00:00 to 2025-03-06 05:27:58.781180+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (5.42s) 


In [19]:
current_date

Timestamp('2025-03-06 06:27:58.781180+0000', tz='Etc/UTC')

In [20]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2025-03-05 07:00:00
1,0,0,0,0,0,1,0,1,0,0,...,0,0,0,0,1,0,0,0,3,2025-03-05 07:00:00
2,4,6,2,3,2,0,1,3,2,1,...,2,5,3,3,0,0,0,1,4,2025-03-05 07:00:00
3,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,6,2025-03-05 07:00:00
4,6,4,2,1,4,1,2,1,2,2,...,1,1,0,2,0,2,0,2,7,2025-03-05 07:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,1,1,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,259,2025-03-05 07:00:00
248,1,4,1,0,3,2,5,0,1,2,...,1,1,0,0,0,1,2,0,260,2025-03-05 07:00:00
249,8,15,14,12,23,23,25,28,23,29,...,13,4,1,0,0,0,2,0,261,2025-03-05 07:00:00
250,151,147,158,114,84,84,65,73,99,89,...,5,1,2,2,4,7,18,79,262,2025-03-05 07:00:00


In [21]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-03-06 01:28:24,903 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-06 01:28:24,925 INFO: Initializing external client
2025-03-06 01:28:24,925 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-06 01:28:26,087 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215672

In [22]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [23]:
predictions

,pickup_location_id,predicted_demand
0,2,0.0
1,3,0.0
2,4,4.0
3,6,0.0
4,7,3.0
...,...,...
247,259,0.0
248,260,1.0
249,261,8.0
250,262,149.0


In [24]:
predictions.sort_values("predicted_demand", ascending=False).head(10)["pickup_location_id"].values

array([236, 186, 237, 239, 162, 262, 238, 142, 141, 140])